In [1]:
￼(ns trenwand.notebook.stats
    (:require [taoensso.encore :as e]
              [clj-time.core :as dt]
              [clj-time.coerce :as dtc]
              [clj-time.format :as dtf]
              [clj-time.predicates :as dtp]
              [clj-time.periodic :as dtper]
              [cuerdas.core :as str]
              [cheshire.core :as json]
              [trennwand.data :as data]
              [trennwand.order :as order]
              [trennwand.color :as color]
              [trennwand.categories :as categories]
              [trennwand.jupyter.echarts :as echarts]
              [net.cgrand.xforms :as x]
              [criterium.core :refer [quick-bench]]
              [clojupyter.misc.display :as display]
              [clojupyter.misc.helper :as helper]
              [clojure.core.reducers :as r]))

CompilerException java.lang.RuntimeException: Unable to resolve symbol: ￼ in this context, compiling:(/tmp/form-init6809589282411486272.clj:1:143) 


class clojure.lang.Compiler$CompilerException: 

In [2]:
(echarts/init)

In [3]:
(def s1-sales (data/read-s1-sales (dt/date-time 2017 7 1) (dt/date-time 2018 9 30)))
(def s3-sales (data/read-s3-sales (dt/date-time 2017 7 1) (dt/date-time 2018 9 30)))

#'trenwand.notebook.stats/s3-sales

In [5]:
(let [s1-report (into [] (comp (filter (fn [{:keys [date]}] 
                                           (and (dt/after? date (dt/date-time 2017 11 1))
                                                (dt/before? date (dt/date-time 2018 4 1)))))
                               (x/by-key :name (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                               (map (fn [[name m]] (assoc m :name name)))
                               (x/sort-by :sales #(compare %2 %1)))
                      s3-sales)
      s3-report (into [] (comp (filter (fn [{:keys [date]}] 
                                           (dt/after? date (dt/date-time 2018 5 1))))
                               (x/by-key :name (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                               (map (fn [[name m]] (assoc m :name name)))
                               (x/sort-by :sales #(compare %2 %1)))
                      s3-sales)
      s1-sum (transduce (map :sales) + s1-report)
      s3-sum (transduce (map :sales) + s3-report)
      s1-percent (into [0] (comp 
                               (map :sales)
                               (partition-all (long (/ (count s1-report) 99))) 
                               (map #(reduce + %))
                               (map #(/ % s1-sum))) 
                           s1-report)
      s3-percent (into [0] (comp 
                               (map :sales)
                               (partition-all (long (/ (count s3-report) 99))) 
                               (map #(reduce + %))
                               (map #(/ % s3-sum))) 
                           s3-report)
      opts {:title {:text "obrót skumulowany"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {:align "right"}
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (map #(e/round2 %) (range 0.0 1.0 0.01))}
                     
            :y-axis {:min 0
                     :max 1}
            :series [{:type "line"
                      :name "->2018.04.01"
                      
                      :data (reductions + s1-percent)}
                     {:type "line"
                      :name "2018.04.01->"
                      
                      :data (reductions + s3-percent)}]}]
    (echarts/plot opts))

In [6]:
(let [s1-report (into [] (comp (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                               (filter (fn [{:keys [date]}] (dt/before? date (dt/date-time 2018 12 31))))
                               (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "1207")))
                               (x/by-key :name (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                               (map (fn [[name m]] (assoc m :name name))))
                      s1-sales)
      s3-report (into [] (comp (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                               (filter (fn [{:keys [date]}] (dt/before? date (dt/date-time 2018 12 31))))
                               (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "1207")))
                               (x/by-key :name (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                               (map (fn [[name m]] (assoc m :name name))))
                      s3-sales)
      s1 (into [] (comp (map #(clojure.set/rename-keys % {:sales :s1}))) s1-report)
      s3 (into [] (comp (map #(clojure.set/rename-keys % {:sales :s3}))) s3-report)
      comb (into [] (x/sort-by :s1 #(compare %2 %1)) (clojure.set/join s1 s3 {:name :name}))
      opts {:title {:text "obrót towarów"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {:data ["s1" "s3"]
                     :align "right"}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 2}
                        {:start 0
                         :end 2}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (map :name comb)}
                     
            :y-axis {:min 0}
            :series [{:type "bar"
                      :name "s1"
                      :data (map :s1 comb)}
                     {:type "bar"
                      :name "s3"
                      :smooth true
                      :data (map :s3 comb)}]}]
    (echarts/plot opts))

In [7]:
(let [s1-report (into [] (comp (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                               (x/by-key :name (x/transjuxt {:profit (comp (map :profit) (x/reduce +))}))
                               (map (fn [[name m]] (assoc m :name name))))
                      s1-sales)
      s3-report (into [] (comp (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                               (x/by-key :name (x/transjuxt {:profit (comp (map :profit) (x/reduce +))}))
                               (map (fn [[name m]] (assoc m :name name))))
                      s3-sales)
      s1 (into [] (comp (map #(clojure.set/rename-keys % {:profit :s1}))) s1-report)
      s3 (into [] (comp (map #(clojure.set/rename-keys % {:profit :s3}))) s3-report)
      comb (into [] (x/sort-by :s1 #(compare %2 %1)) (clojure.set/join s1 s3 {:name :name}))
      opts {:title {:text "zysk towarów"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {:data ["s1" "s3"]
                     :align "right"}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 2}
                        {:start 0
                         :end 2}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (map :name comb)}
                     
            :y-axis {}
            :series [{:type "bar"
                      :name "s1"
                      :data (map :s1 comb)}
                     {:type "bar"
                      :name "s3"
                      :smooth true
                      :data (map :s3 comb)}]}]
    (echarts/plot opts))

In [16]:
(let [s1-report (into [] (comp (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                               (filter (fn [{:keys [sales]}] (pos? sales)))
                               (x/by-key :name (x/transjuxt {:sales (comp (map :sales) (x/reduce +))
                                                             :profit (comp (map :profit) (x/reduce +))}))
                               (map (fn [[name {:keys [sales profit] :as m}]] 
                                        (assoc m :name name
                                                 :margin (/ profit sales)))))
                      s1-sales)
      s3-report (into [] (comp (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                               (filter (fn [{:keys [sales]}] (pos? sales)))
                               (x/by-key :name (x/transjuxt {:sales (comp (map :sales) (x/reduce +))
                                                             :profit (comp (map :profit) (x/reduce +))}))
                               (map (fn [[name {:keys [sales profit] :as m}]] 
                                        (assoc m :name name
                                                 :margin (/ profit sales)))))
                      s3-sales)
      s1 (into [] (comp (map #(clojure.set/rename-keys % {:margin :s1}))) s1-report)
      s3 (into [] (comp (map #(clojure.set/rename-keys % {:margin :s3}))) s3-report)
      comb (into [] (x/sort-by :profit #(compare %2 %1)) (clojure.set/join s1 s3 {:name :name}))
      opts {:title {:text "marża towarów"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {:data ["s1" "s3"]
                     :align "right"}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 2}
                        {:start 0
                         :end 2}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (map :name comb)}
                     
            :y-axis {}
            :series [{:type "bar"
                      :name "s1"
                      :data (map :s1 comb)}
                     {:type "bar"
                      :name "s3"
                      :smooth true
                      :data (map :s3 comb)}]}]
    (echarts/plot opts))

In [9]:
(let [s1-report (into []
                  (comp
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))
                                                    :profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (x/sort-by :date #(compare %1 %2)))
                  s1-sales)
      s1-series (map (fn [{:keys [sales profit date]}]
                      {:value [sales profit]
                       :item-style {:color (color/day-of-week date)}}) s1-report)
      s3-report (into []
                  (comp
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))
                                                    :profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (x/sort-by :date #(compare %1 %2)))
                  s3-sales)
      s3-series (map (fn [{:keys [sales profit date]}]
                      {:value [sales profit]
                       :item-style {:color (color/day-of-week date)}}) s3-report)
      opts {:title {:text "obrót a zysk"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {:data ["s1" "s3"]
                     :align "right"}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 100}
                        {:start 0
                         :end 100}]
            :tooltip {}
            :x-axis {:type "value"
                     :axis-pointer {:show true}}
            :y-axis {}
            :series [{:type "scatter"
                      :name "s1"
                      :symbol-size 8
                      :data s1-series}
                     {:type "scatter"
                      :name "s3"
                      :symbol-size 8
                      :data s3-series}]}]
    (echarts/plot opts))
    

In [10]:
(let [s1-report (into []
                  (comp
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (x/sort-by :date #(compare %1 %2)))
                  s1-sales)
      s1-tma (->> s1-report
                  (reverse)
                  (map :sales)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s1-series (map (fn [{:keys [sales profit date]}]
                      {:value sales
                       :item-style {:color (color/day-of-week date)}}) s1-report)
      s3-report (into []
                  (comp
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (x/sort-by :date #(compare %1 %2)))
                  s3-sales)
      s3-tma (->> s3-report
                  (reverse)
                  (map :sales)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s3-series (map (fn [{:keys [sales profit date]}]
                      {:value sales
                       :item-style {:color (color/day-of-week date)}}) s3-report)
      dates (clojure.set/union (into #{} (map :date) s1-report)
                               (into #{} (map :date) s3-report))
      _ (println (count s1-report) (count s3-report))
      opts {:title {:text "Obrót"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 100}
                        {:start 0
                         :end 100}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (map #(dtc/to-string (:date %)) s1-report)}
            :y-axis {}
            :series [{:type "scatter"
                      :name "s1"
                      :symbol-size 8
                      :data s1-series}
                     {:type "line"
                      :name "s1 srednia"
                      :show-symbol false
                      :data s1-tma}
                     {:type "scatter"
                      :name "s3"
                      :symbol-size 8
                      :data s3-series}
                     {:type "line"
                      :name "s3 srednia"
                      :show-symbol false
                      :data s3-tma}]}]
    (echarts/plot opts))
    

251 251


In [11]:
(let [s1-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/sort-by :date #(compare %1 %2)))
                  s1-sales)
      s1-tma (->> s1-report
                  (reverse)
                  (map :profit)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s1-series (map (fn [{:keys [profit date]}]
                      {:value profit
                       :item-style {:color (color/day-of-week date)}}) s1-report)
      s3-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/sort-by :date #(compare %1 %2)))
                  s3-sales)
      s3-tma (->> s3-report
                  (reverse)
                  (map :profit)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s3-series (map (fn [{:keys [profit date]}]
                      {:value profit
                       :item-style {:color (color/day-of-week date)}}) s3-report)
      opts {:title {:text "zysk"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {:align "right"}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 100}
                        {:start 0
                         :end 100}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (map #(dtc/to-string (:date %)) s1-report)}
            :y-axis {}
            :series [{:type "scatter"
                      :name "s1"
                      :symbol-size 8
                      :data s1-series}
                     {:type "line"
                      :name "s1 srednia"
                      :show-symbol false
                      :data s1-tma}
                     {:type "scatter"
                      :name "s3"
                      :symbol-size 8
                      :data s3-series}
                     {:type "line"
                      :name "s3 srednia"
                      :show-symbol false
                      :data s3-tma}]}]
    (echarts/plot opts))
    

In [12]:
(let [s1-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))
                                                    :profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date {:keys [sales profit] :as m}]] 
                               (assoc m :date date :margin (/ profit sales))))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/sort-by :date #(compare %1 %2)))
                  s1-sales)
      s1-tma (->> s1-report
                  (reverse)
                  (map :margin)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s1-series (map (fn [{:keys [margin date]}]
                      {:value margin
                       :item-style {:color (color/day-of-week date)}}) s1-report)
      s3-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))
                                                    :profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date {:keys [sales profit] :as m}]] 
                               (assoc m :date date :margin (/ profit sales))))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/sort-by :date #(compare %1 %2)))
                  s3-sales)
      s3-tma (->> s3-report
                  (reverse)
                  (map :margin)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s3-series (map (fn [{:keys [margin date]}]
                      {:value margin
                       :item-style {:color (color/day-of-week date)}}) s3-report)
      opts {:title {:text "marża"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {:align "right"}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 100}
                        {:start 0
                         :end 100}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (map #(dtc/to-string (:date %)) s1-report)}
            :y-axis {}
            :series [{:type "scatter"
                      :name "s1"
                      :symbol-size 8
                      :data s1-series}
                     {:type "line"
                      :name "s1 srednia"
                      :show-symbol false
                      :data s1-tma}
                     {:type "scatter"
                      :name "s3"
                      :symbol-size 8
                      :data s3-series}
                     {:type "line"
                      :name "s3 srednia"
                      :show-symbol false
                      :data s3-tma}]}]
    (echarts/plot opts))
    

In [13]:
(let [s1-report (into [] 
                      (comp
                        (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                        (filter #(= "Pieczywo regionalne - Chleb biały" (:category %)))
                        (x/by-key :date (x/transjuxt {:qty (comp (map :qty) (x/reduce +))}))
                        (map (fn [[date m]] (assoc m :date date)))
                        (x/sort-by :date))
                   s1-sales)
      s1-white (map (fn [{:keys [qty date]}]
                      {:value qty
                       :item-style {:color (color/day-of-week date)}}) s1-report)
      s1-report (into [] 
                      (comp
                        (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                        (filter #(= "Pieczywo regionalne - Chleb ciemny" (:category %)))
                        (x/by-key :date (x/transjuxt {:qty (comp (map :qty) (x/reduce +))}))
                        (map (fn [[date m]] (assoc m :date date)))
                        (x/sort-by :date))
                   s1-sales)
      s1-dark (map (fn [{:keys [qty date]}]
                      {:value qty
                       :item-style {:color (color/day-of-week-dark date)}}) s1-report)
      s3-report (into [] 
                      (comp
                        (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                        (filter #(= "Pieczywo regionalne - Chleb biały" (:category %)))
                        (x/by-key :date (x/transjuxt {:qty (comp (map :qty) (x/reduce +))}))
                        (map (fn [[date m]] (assoc m :date date)))
                        (x/sort-by :date))
                   s3-sales)
      s3-white (map (fn [{:keys [qty date]}]
                      {:value qty
                       :item-style {:color (color/day-of-week date)}}) s3-report)
      s3-report (into [] 
                      (comp
                        (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                        (filter #(= "Pieczywo regionalne - Chleb ciemny" (:category %)))
                        (x/by-key :date (x/transjuxt {:qty (comp (map :qty) (x/reduce +))}))
                        (map (fn [[date m]] (assoc m :date date)))
                        (x/sort-by :date))
                   s3-sales)
      s3-dark (map (fn [{:keys [qty date]}]
                      {:value qty
                       :item-style {:color (color/day-of-week-dark date)}}) s3-report)
      dates (clojure.set/union (into #{} (map :date) s1-report)
                               (into #{} (map :date) s3-report))
      opts {:title {:text "ilość chleba białego"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 100}
                        {:start 0
                         :end 100}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (into [] (comp (x/sort) (map #(dtc/to-string %))) dates)}
            :y-axis {:axis-pointer {:show true}}
            :series [{:type "scatter"
                      :name "s1 białe"
                      :symbol-size 8
                      :data s1-white}
                     {:type "scatter"
                      :name "s1 ciemne"
                      :symbol-size 8
                      :data s1-dark}
                     {:type "scatter"
                      :name "s3 białe"
                      :symbol-size 8
                      :data s3-white}
                     {:type "scatter"
                      :name "s3 ciemne"
                      :symbol-size 8
                      :data s3-dark}]}]
    (echarts/plot opts))
    

In [14]:
(let [s1-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "04")))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/sort-by :date #(compare %1 %2)))
                  s1-sales)
      s1-tma (->> s1-report
                  (reverse)
                  (map :sales)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s1-series (map (fn [{:keys [sales profit date]}]
                      {:value sales
                       :item-style {:color (color/day-of-week date)}}) s1-report)
      s3-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "04")))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/sort-by :date #(compare %1 %2)))
                  s3-sales)
      s3-tma (->> s3-report
                  (reverse)
                  (map :sales)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s3-series (map (fn [{:keys [sales profit date]}]
                      {:value sales
                       :item-style {:color (color/day-of-week date)}}) s3-report)
      dates (clojure.set/union (into #{} (map :date) s1-report)
                               (into #{} (map :date) s3-report))
      opts {:title {:text "mięso, wędliny i inne lada - obrót"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 100}
                        {:start 0
                         :end 100}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (map #(dtc/to-string (:date %)) s1-report)}
            :y-axis {:axis-pointer {:show true}}
            :series [{:type "scatter"
                      :name "s1"
                      :symbol-size 8
                      :data s1-series}
                     {:type "line"
                      :name "s1 srednia"
                      :show-symbol false
                      :data s1-tma}
                     {:type "scatter"
                      :name "s3"
                      :symbol-size 8
                      :data s3-series}
                     {:type "line"
                      :name "s3 srednia"
                      :show-symbol false
                      :data s3-tma}]}]
    (echarts/plot opts))
    

In [15]:
(let [s1-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "04")))
                      (x/by-key :date (x/transjuxt {:profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/sort-by :date #(compare %1 %2)))
                  s1-sales)
      s1-tma (->> s1-report
                  (reverse)
                  (map :profit)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s1-series (map (fn [{:keys [sales profit date]}]
                      {:value profit
                       :item-style {:color (color/day-of-week date)}}) s1-report)
      s3-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "04")))
                      (x/by-key :date (x/transjuxt {:profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/sort-by :date #(compare %1 %2)))
                  s3-sales)
      s3-tma (->> s3-report
                  (reverse)
                  (map :profit)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s3-series (map (fn [{:keys [sales profit date]}]
                      {:value profit
                       :item-style {:color (color/day-of-week date)}}) s3-report)
      dates (clojure.set/union (into #{} (map :date) s1-report)
                               (into #{} (map :date) s3-report))
      opts {:title {:text "mięso, wędliny i inne lada - zysk"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 100}
                        {:start 0
                         :end 100}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (map #(dtc/to-string (:date %)) s1-report)}
            :y-axis {:axis-pointer {:show true}}
            :series [{:type "scatter"
                      :name "s1"
                      :symbol-size 8
                      :data s1-series}
                     {:type "line"
                      :name "s1 srednia"
                      :show-symbol false
                      :data s1-tma}
                     {:type "scatter"
                      :name "s3"
                      :symbol-size 8
                      :data s3-series}
                     {:type "line"
                      :name "s3 srednia"
                      :show-symbol false
                      :data s3-tma}]}]
    (echarts/plot opts))
    

In [16]:
(let [s1-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "09")))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/sort-by :date #(compare %1 %2)))
                  s1-sales)
      s1-tma (->> s1-report
                  (reverse)
                  (map :sales)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s1-series (map (fn [{:keys [sales profit date]}]
                      {:value sales
                       :item-style {:color (color/day-of-week date)}}) s1-report)
      s3-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "09")))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] 
                               (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 10 1))))
                      (x/sort-by :date #(compare %1 %2)))
                  s3-sales)
      s3-tma (->> s3-report
                  (reverse)
                  (map :sales)
                  (partition-all 30 1)
                  (map #(/ (reduce + %) (count %)))
                  (partition-all 12 1)
                  (map #(/ (reduce + %) (count %)))
                  (reverse))
      s3-series (map (fn [{:keys [sales profit date]}]
                      {:value sales
                       :item-style {:color (color/day-of-week date)}}) s3-report)
      dates (clojure.set/union (into #{} (map :date) s1-report)
                               (into #{} (map :date) s3-report))
      opts {:title {:text "owoce i warzywa - obrót"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 100}
                        {:start 0
                         :end 100}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (map #(dtc/to-string (:date %)) s1-report)}
            :y-axis {}
            :series [{:type "scatter"
                      :name "s1"
                      :symbol-size 8
                      :data s1-series}
                     {:type "line"
                      :name "s1 srednia"
                      :show-symbol false
                      :data s1-tma}
                     {:type "scatter"
                      :name "s3"
                      :symbol-size 8
                      :data s3-series}
                     {:type "line"
                      :name "s3 srednia"
                      :show-symbol false
                      :data s3-tma}]}]
    (echarts/plot opts))
    

In [17]:
(let [s1-report (into [] 
                      (comp
                        (x/by-key :date (x/transjuxt {:sales    (comp (map :sales) (x/reduce +))
                                                      :r1       (comp (filter #(= "Pieczywo regionalne - Chleb biały" (:category %)))
                                                                      (map :qty)
                                                                      (x/reduce +))
                                                      :r2       (comp (filter #(= "Pieczywo regionalne - Chleb ciemny" (:category %)))
                                                                      (map :qty)
                                                                      (x/reduce +))}))
                        (map (fn [[date m]] (assoc m :date date))))
                   s1-sales)
      s3-report (into [] 
                      (comp
                        (x/by-key :date (x/transjuxt {:sales    (comp (map :sales) (x/reduce +))
                                                      :r1       (comp (filter #(= "Pieczywo regionalne - Chleb biały" (:category %)))
                                                                      (map :qty)
                                                                      (x/reduce +))
                                                      :r2       (comp (filter #(= "Pieczywo regionalne - Chleb ciemny" (:category %)))
                                                                      (map :qty)
                                                                      (x/reduce +))}))
                        (map (fn [[date m]] (assoc m :date date))))
                   s3-sales)
      [[min-sales max-sales]] (sequence (comp (map :sales) (x/transjuxt [x/min x/max])) s1-report)
      [offset slope] (transduce identity (kixi.stats.core/simple-linear-regression :sales :r1) s1-report)
      s1-line (map (juxt identity #(+ (* % slope) offset)) (range min-sales (+ max-sales 100) 100))
      s1-series (map (fn [{:keys [sales r1 date]}]
                      {:value [sales r1]
                       :item-style {:color (color/day-of-week date)}}) s1-report)
      [[min-sales max-sales]] (sequence (comp (map :sales) (x/transjuxt [x/min x/max])) s3-report)
      [offset slope] (transduce identity (kixi.stats.core/simple-linear-regression :sales :r1) s3-report)
      s3-line (map (juxt identity #(+ (* % slope) offset)) (range min-sales (+ max-sales 100) 100))
      s3-series (map (fn [{:keys [sales r1 date]}]
                      {:value [sales r1]
                       :item-style {:color (color/day-of-week date)}}) s3-report)
      opts {:title {:text "sprzedaż pieczywa a obrót"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {:data ["s1" "s1 regresja" "s3" "s3 regresja"]
                     :align "right"}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 100}
                        {:start 0
                         :end 100}]
            :tooltip {}
            :x-axis {:type "value"
                     :axis-pointer {:show true}}
            :y-axis {:axis-pointer {:show true}}
            :series [{:type "scatter"
                      :name "s1"
                      :symbol-size 8
                      :data s1-series}
                     {:type "line"
                      :name "s1 regresja"
                      :show-symbol false
                      :data s1-line}
                     {:type "scatter"
                      :name "s3"
                      :symbol-size 8
                      :data s3-series}
                     {:type "line"
                      :name "s3 regresja"
                      :show-symbol false
                      :data s3-line}]}]
    (echarts/plot opts))

In [17]:
(def viridis
    ["#440154"
"#481567"
"#482677"
"#453781"
"#404788"
"#39568C"
"#33638D"
"#2D708E"
"#287D8E"
"#238A8D"
"#1F968B"
"#20A387"
"#29AF7F"
"#3CBB75"
"#55C667"
"#73D055"
"#95D840"
"#B8DE29"
"#DCE319"
"#FDE725"])

#'trenwand.notebook.stats/viridis

In [18]:
(let [s1-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s1-sales)
      s1-series (map (fn [{:keys [sales date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) sales]) s1-report)
      s3-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s3-sales)
      s3-series (map (fn [{:keys [sales date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) sales]) s3-report)
      opts {:title {:text "obrót"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :visual-map {:min 1000
                         :max 12000
                         :item-height 250
                         :color [color/blue-5 color/green-5 color/gold-5 color/red-5]
                         :calculable true
                         :orient "horizontal"
                         :left "center"
                         :top "top"}
            :calendar [{:range ["2017-07-01" "2018-06-30"]
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :year-label {:formatter "s1"}
                        :cell-size ["auto" 20]}
                       {:range ["2017-07-01" "2018-06-30"]
                        :top "60%"
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :year-label {:formatter "s3"}
                        :cell-size ["auto" 20]}]
            :series [{:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 0
                      :data s1-series}
                     {:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 1
                      :data s3-series}]}]
    (echarts/plot opts))
    

In [19]:
(let [s1-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s1-sales)
      s1-series (map (fn [{:keys [profit date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) profit]) s1-report)
      s3-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s3-sales)
      s3-series (map (fn [{:keys [profit date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) profit]) s3-report)
      opts {:title {:text "zysk"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :visual-map {:min 0
                         :max 2500
                         :item-height 250
                         :color [color/blue-5 color/green-5 color/yelow-5 color/red-5]
                         :calculable true
                         :orient "horizontal"
                         :left "center"
                         :top "top"}
            :calendar [{:range ["2017-07-01" "2018-06-30"]
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :year-label {:formatter "s1"}
                        :cell-size ["auto" 20]}
                       {:range ["2017-07-01" "2018-06-30"]
                        :top "60%"
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :year-label {:formatter "s3"}
                        :cell-size ["auto" 20]}]
            :series [{:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 0
                      :data s1-series}
                     {:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 1
                      :data s3-series}]}]
    (echarts/plot opts))
    

In [20]:
(let [s1-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))
                                                    :profit (comp (map :profit) (x/reduce +))}))
                      (filter (fn [[date {:keys [sales]}]] (pos? sales)))
                      (map (fn [[date {:keys [sales profit] :as m}]] 
                               (assoc m :date date :margin (/ profit sales))))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s1-sales)
      s1-series (map (fn [{:keys [margin date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) margin]) s1-report)
      s3-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))
                                                    :profit (comp (map :profit) (x/reduce +))}))
                      (filter (fn [[date {:keys [sales]}]] (pos? sales)))
                      (map (fn [[date {:keys [sales profit] :as m}]] 
                               (assoc m :date date :margin (/ profit sales))))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s3-sales)
      s3-series (map (fn [{:keys [margin date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) margin]) s3-report)
      opts {:title {:text "marża"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :visual-map {:min 0.1
                         :max 0.2
                         :precision 3
                         :item-height 250
                         :color [color/blue-5 color/green-5 color/yelow-5 color/red-5]
                         :calculable true
                         :orient "horizontal"
                         :left "center"
                         :top "top"}
            :calendar [{:range ["2017-07-01" "2018-06-30"]
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :cell-size ["auto" 20]}
                       {:range ["2017-07-01" "2018-06-30"]
                        :top "60%"
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :cell-size ["auto" 20]}]
            :series [{:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 0
                      :data s1-series}
                     {:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 1
                      :data s3-series}]}]
    (echarts/plot opts))
    

In [21]:
(let [s1-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:white (comp (filter #(= "Pieczywo regionalne - Chleb biały" (:category %)))
                                                                 (map :qty)
                                                                 (x/reduce +))
                                                    :dark (comp (filter #(= "Pieczywo regionalne - Chleb ciemny" (:category %)))
                                                                (map :qty) 
                                                                (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s1-sales)
      s1-white (map (fn [{:keys [white date]}]
                     [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) white]) s1-report)
      s1-dark (map (fn [{:keys [dark date]}]
                    [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) dark]) s1-report)
      s3-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:white (comp (filter #(= "Pieczywo regionalne - Chleb biały" (:category %)))
                                                                 (map :qty)
                                                                 (x/reduce +))
                                                    :dark (comp (filter #(= "Pieczywo regionalne - Chleb ciemny" (:category %)))
                                                                (map :qty) 
                                                                (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s3-sales)
      s3-white (map (fn [{:keys [white date]}]
                     [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) white]) s3-report)
      s3-dark (map (fn [{:keys [dark date]}]
                    [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) dark]) s3-report)
      opts {:height 800
            :title {:text "chleb"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :visual-map {:min 0
                         :max 100
                         :item-height 250
                         :color [color/blue-5 color/green-5 color/yelow-5 color/red-5]
                         :calculable true
                         :orient "horizontal"
                         :left "center"
                         :top "top"}
            :calendar [{:range ["2017-07-01" "2018-06-30"]
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :year-label {:formatter "s1 jasny"}
                        :cell-size ["auto" 20]}
                       {:range ["2017-07-01" "2018-06-30"]
                        :top "250px"
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :year-label {:formatter "s1 ciemny"}
                        :cell-size ["auto" 20]}
                       {:range ["2017-07-01" "2018-06-30"]
                        :top "450px"
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :year-label {:formatter "s3 jasny"}
                        :cell-size ["auto" 20]}
                       {:range ["2017-07-01" "2018-06-30"]
                        :top "650px"
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :year-label {:formatter "s3 ciemny"}
                        :cell-size ["auto" 20]}]
            :series [{:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 0
                      :data s1-white}
                     {:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 1
                      :data s1-dark}
                     {:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 2
                      :data s3-white}
                     {:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 3
                      :data s3-dark}]}]
    (echarts/plot opts))
    

In [22]:
(let [s1-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "04")))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s1-sales)
      s1-series (map (fn [{:keys [sales date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) sales]) s1-report)
      s3-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "04")))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s3-sales)
      s3-series (map (fn [{:keys [sales date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) sales]) s3-report)
      opts {:title {:text "mięso, wędliny i inne lada"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :visual-map {:min 0
                         :max 3500
                         :item-height 250
                         :color [color/blue-5 color/green-5 color/yelow-5 color/red-5]
                         :calculable true
                         :orient "horizontal"
                         :left "center"
                         :top "top"}
            :calendar [{:range ["2017-07-01" "2018-06-30"]
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :cell-size ["auto" 20]}
                       {:range ["2017-07-01" "2018-06-30"]
                        :top "60%"
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :cell-size ["auto" 20]}]
            :series [{:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 0
                      :data s1-series}
                     {:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 1
                      :data s3-series}]}]
    (echarts/plot opts))
    

In [23]:
(let [s1-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "04")))
                      (x/by-key :date (x/transjuxt {:profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s1-sales)
      s1-series (map (fn [{:keys [profit date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) profit]) s1-report)
      s3-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "04")))
                      (x/by-key :date (x/transjuxt {:profit (comp (map :profit) (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s3-sales)
      s3-series (map (fn [{:keys [profit date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) profit]) s3-report)
      opts {:title {:text "mięso, wędliny i inne lada - zysk"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :visual-map {:min 0
                         :max 500
                         :item-height 250
                         :color [color/blue-5 color/green-5 color/yelow-5 color/red-5]
                         :calculable true
                         :orient "horizontal"
                         :left "center"
                         :top "top"}
            :calendar [{:range ["2017-07-01" "2018-06-30"]
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :cell-size ["auto" 20]}
                       {:range ["2017-07-01" "2018-06-30"]
                        :top "60%"
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :cell-size ["auto" 20]}]
            :series [{:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 0
                      :data s1-series}
                     {:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 1
                      :data s3-series}]}]
    (echarts/plot opts))
    

In [24]:
Created new window in existing browser session.(let [s1-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "09")))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s1-sales)
      s1-series (map (fn [{:keys [sales date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) sales]) s1-report)
      s3-report (into []
                  (comp
                      (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "09")))
                      (x/by-key :date (x/transjuxt {:sales (comp (map :sales) (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s3-sales)
      s3-series (map (fn [{:keys [sales date]}]
                      [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) sales]) s3-report)
      max-val (max (apply max (map :sales s1-report)) (apply max (map :sales s3-report)))
      opts {:title {:text "alkohole"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :visual-map {:min 0
                         :max max-val
                         :item-height 250
                         :color [color/blue-5 color/green-5 color/yelow-5 color/red-5]
                         :calculable true
                         :orient "horizontal"
                         :left "center"
                         :top "top"}
            :calendar [{:range ["2017-07-01" "2018-06-30"]
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :cell-size ["auto" 20]}
                       {:range ["2017-07-01" "2018-06-30"]
                        :top "60%"
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :cell-size ["auto" 20]}]
            :series [{:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 0
                      :data s1-series}
                     {:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 1
                      :data s3-series}]}]
    (echarts/plot opts))
    

CompilerException java.lang.RuntimeException: Unable to resolve symbol: Created in this context, compiling:(/tmp/form-init5343416641844511807.clj:1:143) 
CompilerException java.lang.RuntimeException: Unable to resolve symbol: new in this context, compiling:(/tmp/form-init5343416641844511807.clj:1:143) 
CompilerException java.lang.RuntimeException: Unable to resolve symbol: window in this context, compiling:(/tmp/form-init5343416641844511807.clj:1:143) 
CompilerException java.lang.RuntimeException: Unable to resolve symbol: in in this context, compiling:(/tmp/form-init5343416641844511807.clj:1:143) 
CompilerException java.lang.RuntimeException: Unable to resolve symbol: existing in this context, compiling:(/tmp/form-init5343416641844511807.clj:1:143) 
CompilerException java.lang.RuntimeException: Unable to resolve symbol: browser in this context, compiling:(/tmp/form-init5343416641844511807.clj:1:143) 
CompilerException java.lang.ClassNotFoundException: session., compiling:(/tmp/form-in

class clojure.lang.Compiler$CompilerException: 

In [25]:
(defn treemap
  ([key idx coll]
   (when idx
     (e/assoc-some
       {:name (get categories/raw-categories idx)
        :value (long (transduce (comp (filter #(str/starts-with? (:category-id %) idx))
                                 (map key))
                          + coll))}
       :children (not-empty
                   (mapv (fn [[k v]]
                           (treemap key k coll))
                         (categories/ancestors idx))))))
  ([key coll]
   (mapv (fn [[k v]] (treemap key k coll)) categories/parents)))

#'trenwand.notebook.stats/treemap

In [26]:
(def s1-tree-sales (treemap :sales (into [] (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2018 5 1)))) s1-sales)))
(def s3-tree-sales (treemap :sales (into [] (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2018 5 1)))) s3-sales)))
(def s1-tree-profit (treemap :profit (into [] (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2018 5 1)))) s1-sales)))
(def s3-tree-profit (treemap :profit (into [] (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2018 5 1)))) s3-sales)))

#'trenwand.notebook.stats/s3-tree-profit

In [27]:
(let [opts {:title {:text "sklep 1 - obrót"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :series [{:type "treemap"
                      :name "obrót"
                      :roam false
                      :leaf-depth 2
                      :data s1-tree-sales
                      :levels [{:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}]}]}]
    (echarts/plot opts))

In [28]:
(let [opts {:title {:text "sklep 3 - obrót"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :series [{:type "treemap"
                      :name "obrót"
                      :roam false
                      :leaf-depth 2
                      :data s3-tree-sales
                      :levels [{:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}]}]}]
    (echarts/plot opts))

In [29]:
(let [opts {:title {:text "sklep 1 - zysk"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :series [{:type "treemap"
                      :name "zysk"
                      :roam false
                      :leaf-depth 1
                      :data s1-tree-profit
                      :levels [{:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}]}]}]
    (echarts/plot opts))

In [30]:
(let [opts {:title {:text "sklep 3 - zysk"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :series [{:type "treemap"
                      :name "zysk"
                      :roam false
                      :leaf-depth 1
                      :data s3-tree-profit
                      :levels [{:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}
                               {:item-style {:normal {:border-color "#000"
                                                      :border-width 1}}}]}]}]
    (echarts/plot opts))

In [31]:
(first s1-sales)

{:category "Przekąski", :sales 3.52, :date #object[org.joda.time.DateTime 0x5c343db3 "2017-08-21T00:00:00.000Z"], :name "CHRUPKI CHEETOS PIZZA 43G", :ean "5900259087140", :profit 0.72, :category-id "120101", :weight "0.043", :id "000077", :qty 2.0}

In [32]:
(defn make-series [key coll]
  (into [] (comp 
            (filter (fn [{:keys [category-id]}] (str/starts-with? category-id "09")))
            (x/by-key (fn [{:keys [category-id date]}] 
                        [(str/slice category-id 0 6)
                         (dt/first-day-of-the-month date)]) 
                      (comp (map key) (x/reduce +)))
            (map (fn [[[c d] sales]] 
                   [(dtc/to-string d) 
                    (long sales)
                    (get categories/raw-categories c c)])))
             
        coll))

#'trenwand.notebook.stats/make-series

In [33]:
(let [opts {:height 600
            :title {:text "sklep 1"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :data-zoom [{:type "inside"
                         :start 10
                         :end 90}
                        {:start 10
                         :end 90}]
            :tooltip {:trigger "axis"}
            :single-axis {:type "time"}
            :axis-pointer {:label {:show true}}
            :series [{:type "themeRiver"
                      :name "s1"
                      :data (make-series :sales s1-sales)}]}]
    (echarts/plot opts))

In [34]:
{:axis-pointer {:show true}}(let [opts {:height 600
                                        :title {:text "sklep 3"}
                                        :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
                                        :legend {}
                                        :data-zoom [{:type "inside"
                                                     :start 10
                                                     :end 95}
                                                    {:start 0
                                                     :end 95}]
                                        :tooltip {:trigger "axis"}
                                        :single-axis {:type "time"}
                                        :axis-pointer {:label {:show true}}
                                        :series [{:type "themeRiver"
                                                  :name "s1"
                                                  :data (make-series :sales s3-sales)}]}]
                                (echarts/plot opts))

# Porównanie towaru

In [35]:
(defn make-comparision-series [sales [from to] [from1 to1] {:keys [category-id
                                                                   method
                                                                   diff?]
                                                            :or {method :sales}}]
  (let [r1 (into [] (comp (filter (fn [{:keys [date]}] (dt/after? date from)))
                          (filter (fn [{:keys [date]}] (dt/before? date to)))
                          (filter (fn [{id :category-id :as m}] 
                                    (if category-id 
                                      (str/starts-with? id category-id)
                                      (identity m))))
                          (x/by-key :name (x/transjuxt {:value (comp (map method) (x/reduce +))}))
                          (map (fn [[name m]] (assoc m :name name))))
                 sales)
        r2 (into [] (comp (filter (fn [{:keys [date]}] (dt/after? date from1)))
                          (filter (fn [{:keys [date]}] (dt/before? date to1)))
                          (filter (fn [{id :category-id :as m}] 
                                    (if category-id 
                                      (str/starts-with? id category-id)
                                      (identity m))))
                          (x/by-key :name (x/transjuxt {:value (comp (map method) (x/reduce +))}))
                          (map (fn [[name m]] (assoc m :name name))))
                 sales)
        find-value (fn [name coll]
                     (let [[m] (sequence (filter (fn [{x :name}] (= x name))) coll)]
                       (get m :value 0.0)))
        names (distinct (into (mapv :name r1) (mapv :name r2)))
        series (into []
                     (comp
                      (map (fn [name]
                             {:name name
                              :r1 (find-value name r1)
                              :r2 (find-value name r2)
                              :diff (- (find-value name r1) (find-value name r2))}))
                      (x/sort-by :r1 #(compare %1 %2)))
                     names)]
      series))

#'trenwand.notebook.stats/make-comparision-series

In [36]:
(let [series (make-comparision-series s1-sales 
                                      [(dt/date-time 2018 4 1) (dt/date-time 2018 5 1)]
                                      [(dt/date-time 2017 10 1) (dt/date-time 2017 11 1)]
                                      {})
                                      
      opts {:title {:text "różnica okresów"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {:data ["okres 1" "okres 2" "roznica"]
                     :align "right"}
            :data-zoom [{:type "inside"
                         :start 0
                         :end 100}
                        {:start 0
                         :end 100}]
            :tooltip {}
            :x-axis {:axis-pointer {:show true}
                     :data (take 200 (map :name series))}
                     
            :y-axis {}
            :series [{:type "bar"
                      :name "okres 1"
                      :data (take 200 (map :r1 series))}
                     {:type "bar"
                      :name "okres 2"
                      :data (take 200 (map :r2 series))}
                     {:type "bar"
                      :name "roznica"
                      :data (take 200 (map :diff series))}]}]
    (echarts/plot opts))

In [37]:
(let [s1-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:sales (comp (filter #(= "060802" (:category-id %)))
                                                                 (map :profit)
                                                                 (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s1-sales)
      s1-series (map (fn [{:keys [sales date]}]
                     [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) sales]) s1-report)
      s3-report (into []
                  (comp
                      (x/by-key :date (x/transjuxt {:sales (comp (filter #(= "060802" (:category-id %)))
                                                                 (map :profit)
                                                                 (x/reduce +))}))
                      (map (fn [[date m]] (assoc m :date date)))
                      (filter (fn [{:keys [date]}] (dt/after? date (dt/date-time 2017 6 1)))))
                  s3-sales)
      s3-series (map (fn [{:keys [sales date]}]
                     [(dtf/unparse (dtf/formatter "yyyy-MM-dd") date) sales]) s3-report)
      opts {:height 800
            :title {:text "jogurty"}
            :toolbox {:feature {:save-as-image {:pixel-ratio 2}}}
            :legend {}
            :tooltip {}
            :visual-map {:min 0
                         :max 40
                         :item-height 250
                         :color [color/blue-5 color/green-5 color/yelow-5 color/red-5]
                         :calculable true
                         :orient "horizontal"
                         :left "center"
                         :top "top"}
            :calendar [{:range ["2017-07-01" "2018-06-30"]
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :year-label {:formatter "s1 jogurty"}
                        :cell-size ["auto" 20]}
                       {:range ["2017-07-01" "2018-06-30"]
                        :top "250px"
                        :day-label {:first-day 1
                                    :name-map ["nd" "pn" "wt" "śr" "cz" "pt" "so"]}
                        :month-label {:name-map ["sty" "lut" "mar" "kwi" "maj" "cze"
                                                 "lip" "sie" "wrz" "paź" "lis" "gru"]}
                        :year-label {:formatter "s3 jogurty"}
                        :cell-size ["auto" 20]}]
            :series [{:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 0
                      :data s1-series}
                     {:type "heatmap"
                      :coordinateSystem "calendar"
                      :calendar-index 1
                      :data s3-series}]}]
    (echarts/plot opts))
    